In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('data/train.csv')
target = pd.read_csv('data/train_target.csv')

In [ ]:

plt.style.use('ggplot')


%matplotlib inline
plt.rcParams['figure.figsize'] = (12,8)

pd.options.mode.chained_assignment = None



# чтение данных
df = pd.read_csv('data/train.csv')
target = pd.read_csv('data/train_target.csv')

# shape and data types of the data
print(df.shape)
print(df.dtypes)

# отбор числовых колонок
df_numeric = df.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values
print(numeric_cols)

In [ ]:
cols = df.columns 

colours = ['#000000', '#ffff00'] 
sns.heatmap(df[cols].isnull(), cmap=sns.color_palette(colours))

In [ ]:
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing)))

In [ ]:
df['mes_id'] = df['wagnum'].apply(str) + '_' + df['ts_id'].apply(str)

In [ ]:
# df

In [ ]:
target['mes_id'] = target['wagnum'].apply(str) + '_' + target['ts_id'].apply(str)

In [ ]:
target

In [ ]:
merged_df = pd.merge(df, target, on='mes_id')

In [ ]:
merged_df

In [ ]:
merged_df = merged_df.drop(['wagnum_y', 'ts_id_y'], axis=1)

In [ ]:
merged_df.columns = ['wagnum', 'ts_id', 'milleage_all', 'axl1_l_w_flange',
       'axl1_r_w_flange', 'axl2_l_w_flange', 'axl2_r_w_flange',
       'axl3_l_w_flange', 'axl3_r_w_flange', 'axl4_l_w_flange',
       'axl4_r_w_flange', 'mes_id', 'target']

In [ ]:
# merged_df.to_csv('train_with_target.csv', index=False)

In [ ]:
merged_df = pd.read_csv('data/train_with_target.csv')

In [ ]:
merged_df['target'].hist(bins=100)

In [ ]:
merged_df = merged_df.sort_values(by='target')
merged_df.head()

In [ ]:
measures = ['axl1_l_w_flange', 'axl1_r_w_flange', 'axl2_l_w_flange', 
'axl2_r_w_flange', 'axl3_l_w_flange', 'axl3_r_w_flange', 'axl4_l_w_flange', 'axl4_r_w_flange']

In [ ]:
#for m in measures:
m = "axl2_l_w_flange"
plt.scatter(merged_df[merged_df['mes_id']=="37156_2"]['milleage_all'], merged_df[merged_df['mes_id']=="37156_2"][m])#, color='C0')#,label = f'Вагон А{m}')
plt.scatter(merged_df[merged_df['mes_id']=="25397_2"]['milleage_all'], merged_df[merged_df['mes_id']=="25397_2"][m])#, color='C0')#,label = f'Вагон А{m}')
plt.scatter(merged_df[merged_df['mes_id']=="13609_5"]['milleage_all'], merged_df[merged_df['mes_id']=="13609_5"][m])#, color='C0')#,label = f'Вагон А{m}')

#    plt.plot(merged_df[merged_df['mes_id']==w1]['milleage_all'], merged_df[merged_df['mes_id']==w1][m], color='C1')#,label = f'Вагон Б{m}')
plt.xlabel("Пройденный путь")
plt.ylabel("Размер фичи")
plt.title("Зависимость размеров двух вагонов от пройденного пути")
plt.grid()
plt.legend()

## Решение №1. 
### Для каждого вагона сделаем фичи: угол наклона прямой тенденции изменения размера текущей фичи и ее сдвиг. Т.е обучаем линейную регрессию берем из нее коэфициенты и записываем таблицу.

In [ ]:
merged_df = pd.read_csv('data/train_with_target.csv')
merged_df

In [ ]:
measures = ['axl1_l_w_flange', 'axl1_r_w_flange', 'axl2_l_w_flange', 
'axl2_r_w_flange', 'axl3_l_w_flange', 'axl3_r_w_flange', 'axl4_l_w_flange', 'axl4_r_w_flange']

In [ ]:
from sklearn.linear_model import LinearRegression
lg = LinearRegression()
x = merged_df[merged_df['mes_id'] == '2_3']["milleage_all"].to_numpy().reshape(-1,1)
y = merged_df[merged_df['mes_id'] == '2_3']["axl1_l_w_flange"]
lg.fit(x,y)
a = lg.coef_#.predict(x[0,:])#,lg.predict(x[-1])
b = lg.intercept_
plt.plot(x,y,x,x*a+b)

In [ ]:
from sklearn.linear_model import LinearRegression
lg = LinearRegression()
x = merged_df[merged_df['mes_id'] == '2_3']["milleage_all"].to_numpy().reshape(-1,1)
y = merged_df[merged_df['mes_id'] == '2_3']["target"].to_numpy().reshape(-1,1)
y = np.max(x) - x + y
lg.fit(x,y)
a = lg.coef_#.predict(x[0,:])#,lg.predict(x[-1])
b = lg.intercept_
plt.plot(x,y,x,x*a+b);

In [ ]:
merged_df[merged_df['mes_id'] == '2_3']["milleage_all"]

In [ ]:
import tqdm

In [ ]:



%%time
lin_regr_dict = {}
lin_regr_dict["mes_id"] = []
lin_regr_dict["target"] = []
lin_regr_dict["wagnum"] = []
lin_regr_dict["ts_id"] = []
lin_regr_dict["milleage_all"] = []

for i in measures:
    lin_regr_dict[i+"_"+"a"] = []
    lin_regr_dict[i+"_"+"b"] = []
    
for m_id in tqdm.tqdm(merged_df["mes_id"].unique()):
    lin_regr_dict["mes_id"].append(m_id)
    lin_regr_dict["target"].append(merged_df[merged_df["mes_id"] ==m_id]["target"].max())
    lin_regr_dict["wagnum"].append(merged_df[merged_df["mes_id"] ==m_id]["wagnum"].max())
    lin_regr_dict["ts_id"].append(merged_df[merged_df["mes_id"] ==m_id]["ts_id"].max())
    lin_regr_dict["milleage_all"].append(merged_df[merged_df["mes_id"] ==m_id]["milleage_all"].max())

    for m in measures:
        lg = LinearRegression(n_jobs = -1)
        x = merged_df[merged_df['mes_id'] == m_id]["milleage_all"].to_numpy()#.reshape(-1,1)
        y = merged_df[merged_df['mes_id'] == m_id][m].to_numpy()
        not_na_indexes = ~np.isnan(y) & ~np.isnan(x)
        lg.fit(x[not_na_indexes].reshape(-1,1),y[not_na_indexes].reshape(-1,1))

            
        a = lg.coef_
        b = lg.intercept_
        lin_regr_dict[m+"_"+"a"].append(float(a))
        lin_regr_dict[m+"_"+"b"].append(float(b))

In [ ]:
lin_regr_df = pd.DataFrame(lin_regr_dict)
lin_regr_df

In [ ]:
lin_regr_df.to_csv('data/lin_regr_df.csv', index=False)

In [ ]:
#Пробую тоже самое для таргета

lin_regr_dict_t = {}
lin_regr_dict_t["mes_id"] = []
lin_regr_dict_t["target"] = []
lin_regr_dict_t["wagnum"] = []
lin_regr_dict_t["ts_id"] = []
lin_regr_dict_t["milleage_all"] = []

#for i in measures:
lin_regr_dict_t["target_div_all"] = []
    
for m_id in tqdm.tqdm(merged_df["mes_id"].unique()):
    lin_regr_dict_t["mes_id"].append(m_id)
    lin_regr_dict_t["target"].append(merged_df[merged_df["mes_id"] ==m_id]["target"].max())#max используется из-за того,что там все значения одинаковые, соот-но можно брать любое значение
    lin_regr_dict_t["wagnum"].append(merged_df[merged_df["mes_id"] ==m_id]["wagnum"].max())
    lin_regr_dict_t["ts_id"].append(merged_df[merged_df["mes_id"] ==m_id]["ts_id"].max())
    lin_regr_dict_t["milleage_all"].append(merged_df[merged_df["mes_id"] ==m_id]["milleage_all"].max())
    if len(np.unique(merged_df[merged_df["mes_id"] ==m_id]["target"])) != 1:
        print(np.unique(merged_df[merged_df["mes_id"] ==m_id]["milleage_all"]))
    for m in measures:
        lg = LinearRegression(n_jobs = -1)
        x = merged_df[merged_df['mes_id'] == m_id]["milleage_all"].to_numpy()#.reshape(-1,1)
        y = merged_df[merged_df['mes_id'] == m_id][m].to_numpy()
        not_na_indexes = ~np.isnan(y) & ~np.isnan(x)
        lg.fit(x[not_na_indexes].reshape(-1,1),y[not_na_indexes].reshape(-1,1))


        a = lg.coef_
        b = lg.intercept_
        lin_regr_dict_t[m+"_"+"a"].append(float(a))
        lin_regr_dict_t[m+"_"+"b"].append(float(b))


In [ ]:
for m in measures:
    ar = []
    eps = 2

    for index,row in tqdm.tqdm(lin_regr_df.iterrows()):
        #print(row["mes_id"])
        av_target = 0
        current_m_value = row[m]
        mask = (current_m_value-eps<lin_regr_df[m])&(lin_regr_df[m]<current_m_value+eps)
        common_values = lin_regr_df[mask][m]
        ar.append(common_values.mean())
    lin_regr_df["av_target_for_"+m] = ar

In [ ]:
# lin_regr_df.to_csv("lin_regr_df_upd_with_mean_target_feature.csv",index = False)

In [ ]:
lin_regr_df = pd.read_csv('data/lin_regr_df_upd_with_mean_target_feature.csv')

In [ ]:
lin_regr_df.columns

In [ ]:
measures = ['axl1_l_w_flange', 'axl1_r_w_flange', 'axl2_l_w_flange', 
'axl2_r_w_flange', 'axl3_l_w_flange', 'axl3_r_w_flange', 'axl4_l_w_flange', 'axl4_r_w_flange']
for m in measures:
    lin_regr_df[m] = lin_regr_df["milleage_all"] *lin_regr_df[m+"_a"]+lin_regr_df[m+"_b"]
# for i in #lin_regr_df.iterrows():
#     print(i)

In [ ]:
import seaborn as sns
sns.heatmap(lin_regr_df.drop(columns=["mes_id"]).corr())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#Дописать Стандарт Скалер
#scaler = StandardScaler()

y = lin_regr_df["target"]
x = lin_regr_df.drop(columns=["target","mes_id","wagnum"])
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.svm import SVR
params = {
    "objective": "reg:squarederror",
    "n_estimators":100,
    "max_depth": 5,#5
    "eval_metric":'mae'
}
model = xgb.XGBRegressor( **params)#xgb.XGBRegressor(**params )#xgb.XGBRegressor()#GradientBoostingRegressor()#RandomForestRegressor()#LinearRegression()#GradientBoostingRegressor(loss = 'absolute_error')
#model = xgb.XGBRegressor( )#xgb.XGBRegressor()#GradientBoostingRegressor()#RandomForestRegressor()#LinearRegression()#GradientBoostingRegressor(loss = 'absolute_error')

model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mean_absolute_error(np.repeat(y_test,19), np.repeat(y_pred,19))


In [ ]:
y_train_pred = model.predict(x_train)
error = np.abs(y_train_pred - y_train)
error.hist()

In [ ]:
mask = error<30000
model = xgb.XGBRegressor(**params)#RandomForestRegressor()#LinearRegression()#GradientBoostingRegressor(loss = 'absolute_error')
model.fit(x_train[mask],y_train[mask])

y_pred = model.predict(x_test)
y_pred[y_pred<600] = 5000
mean_absolute_error(np.repeat(y_test,19), np.repeat(y_pred,19))


In [ ]:
np.mean(y_pred), np.mean(y_test), np.mean(y_train)

In [ ]:
max(y_train), max(y_test), max(y_pred)

In [ ]:
test_df = pd.read_parquet("./data/test.parquet")
test_df["mes_id"] = test_df["wagnum"].astype(str) + "_" + test_df["ts_id"].astype(str)
test_df

In [ ]:
test_df.to_csv("test.csv",index = False)

In [ ]:
%%time
import tqdm
#То, что закомментировано удалять не нужно. Я закомментил, чтобы файлы не перезаписывались

merged_df = test_df
lin_regr_dict = {}
lin_regr_dict["mes_id"] = []
#lin_regr_dict["target"] = []
lin_regr_dict["wagnum"] = []
lin_regr_dict["ts_id"] = []
lin_regr_dict["milleage_all"] = []

for i in measures:
    lin_regr_dict[i+"_"+"a"] = []
    lin_regr_dict[i+"_"+"b"] = []
    
for m_id in tqdm.tqdm(merged_df["mes_id"].unique()):
    lin_regr_dict["mes_id"].append(m_id)
    #lin_regr_dict["target"].append(merged_df[merged_df["mes_id"] ==m_id]["target"].max())#max используется из-за того,что там все значения одинаковые, соот-но можно брать любое значение
    lin_regr_dict["wagnum"].append(merged_df[merged_df["mes_id"] ==m_id]["wagnum"].max())
    lin_regr_dict["ts_id"].append(merged_df[merged_df["mes_id"] ==m_id]["ts_id"].max())
    lin_regr_dict["milleage_all"].append(merged_df[merged_df["mes_id"] ==m_id]["milleage_all"].max())

    for m in measures:
        
        lg = LinearRegression(n_jobs = -1)
        x = merged_df[merged_df['mes_id'] == m_id]["milleage_all"].to_numpy()#.reshape(-1,1)
        y = merged_df[merged_df['mes_id'] == m_id][m].to_numpy()
        #print(1)
        not_na_indexes = ~np.isnan(y) & ~np.isnan(x)
        #print(2)
        lg.fit(x[not_na_indexes].reshape(-1,1),y[not_na_indexes].reshape(-1,1))
       # print(3)
            
        a = lg.coef_
        b = lg.intercept_
        #print(4)
        lin_regr_dict[m+"_"+"a"].append(float(a))
        lin_regr_dict[m+"_"+"b"].append(float(b))
print(5)
lin_regr_df = pd.DataFrame(lin_regr_dict)

In [ ]:
lin_regr_df

In [ ]:
measures = ['axl1_l_w_flange', 'axl1_r_w_flange', 'axl2_l_w_flange', 
'axl2_r_w_flange', 'axl3_l_w_flange', 'axl3_r_w_flange', 'axl4_l_w_flange', 'axl4_r_w_flange']
for m in measures:
    lin_regr_df[m] = lin_regr_df["milleage_all"] *lin_regr_df[m+"_a"]+lin_regr_df[m+"_b"]

In [ ]:
lin_regr_df.to_csv("test_linregr.csv",index = False)

In [ ]:
for m in measures:
    ar = []
    eps = 2

    for index,row in tqdm.tqdm(lin_regr_df.iterrows()):
        #print(row["mes_id"])
        av_target = 0
        current_m_value = row[m]
        mask = (current_m_value-eps<lin_regr_df[m])&(lin_regr_df[m]<current_m_value+eps)
        common_values = lin_regr_df[mask][m]
        ar.append(common_values.mean())
    lin_regr_df["av_target_for_"+m] = ar
        #print(ar)
        #break
        #ar.append()
    #lin_regr_df["av_target_for_"+m] = lin

In [ ]:
lin_regr_df.to_csv("data/test_linregr_upd_with_av_target_feature.csv",index = False)

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.svm import SVR
params = {
    "objective": "reg:squarederror",
    "n_estimators":100,
    "max_depth": 5,#5
    "eval_metric":'mae'
}
model = xgb.XGBRegressor( **params)#xgb.XGBRegressor(**params )#xgb.XGBRegressor()#GradientBoostingRegressor()#RandomForestRegressor()#LinearRegression()#GradientBoostingRegressor(loss = 'absolute_error')
#model = xgb.XGBRegressor( )#xgb.XGBRegressor()#GradientBoostingRegressor()#RandomForestRegressor()#LinearRegression()#GradientBoostingRegressor(loss = 'absolute_error')

model.fit(x_train,y_train)
y_pred = model.predict(x_test)
mean_absolute_error(np.repeat(y_test,19), np.repeat(y_pred,19))


In [ ]:
y_train_pred = model.predict(x_train)
error = np.abs(y_train_pred - y_train)
error.hist()

In [ ]:
mask = error<30000
model = xgb.XGBRegressor(**params)#RandomForestRegressor()#LinearRegression()#GradientBoostingRegressor(loss = 'absolute_error')
model.fit(x_train[mask],y_train[mask])

y_pred = model.predict(x_test)
y_pred[y_pred<600] = 5000
mean_absolute_error(np.repeat(y_test,19), np.repeat(y_pred,19))


In [ ]:
lin_regr_df = pd.read_csv("data/test_linregr_upd_with_av_target_feature.csv")

In [ ]:
need_order = list(model.feature_names_in_)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#test_subm_y = lin_regr_df["target"]
test_subm_x = lin_regr_df.drop(columns=["mes_id","wagnum"])
test_subm_x = test_subm_x[need_order]

In [ ]:
lin_regr_df["target"] = model.predict(test_subm_x)

In [ ]:
lin_regr_df

In [ ]:
lin_regr_df.loc[:,["wagnum","ts_id","target"]].to_csv("ML ботать нельзя спать.csv")